In [2]:
"""
    @author: Ben, Yusuf, Kendrick
    Trains models to predict disease gene associations.
    """
import sys
sys.path.append('../')

import torch
import torch.nn.functional as F
from datetime import datetime
import numpy as np
from load_assoc import ProcessData
#from neural_net import GNN
#import utils
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
import copy
import random
import pandas as pd
#import conv_layers
#import optimizers




def trainer(args, num_folds=5):
    edgelist_file = args['dataset']

    processed_data = ProcessData(edgelist_file, use_features=args['use_features'])
    X = processed_data.X
    X = torch.tensor(X.values, dtype=torch.float)
    disease_test_scores = {}


    # This returns all disease indices corresponding to given disease classes
    sel_diseases = processed_data.get_disease_class_idx(['cancer'])
    processed_data.Y = processed_data.Y.iloc[:,sel_diseases]

    for ind, column in enumerate(processed_data.Y):
        #if ind > 5: break # TODO: Remove this later on. For testing purposes only
        
        y = processed_data.Y[column].tolist()
        edges = processed_data.get_edges()
        
        y = torch.tensor(np.array(y).astype('int'), dtype=torch.long)
        edges = torch.tensor(edges.values, dtype=torch.long)
        
        # Set up train and test sets:
        test_size = .1
        #data_generator = load_pyg(X, edges, y,
        #                                folds=num_folds, test_size=test_size)
            
        return X, edges, y

if __name__ == '__main__':
    dt = str(datetime.now())[5:19].replace(' ', '_').replace(':', '-')
    
    args={}
    args['network-type']='GCNConv'
    args['dataset']='../dataset_collection/GNBR-edgelist.csv'
    args['expt_name']=dt
    args['use_features']=True

    print('Experiment name is', args['expt_name'])
    x = trainer(args)


Experiment name is 04-07_17-55-22


FileNotFoundError: [Errno 2] No such file or directory: '../Data/NCBI_to_UNIPROT.txt'

In [ ]:
def load_pyg(X, edges, y, folds=5, test_size=0.1):

    # First identify the test set
    indices = np.arange(len(X))
    _,test_idx = train_test_split(indices, test_size=test_size,
                                             stratify=y, random_state=40)

    # Now use the remainder of the data to create train and val sets
    kf = StratifiedKFold(n_splits=folds, random_state=2)

    # Consider all edges and their reverse
    edges = [e for e in edges.numpy() if e[0] != e[1]] # Remove self edges
    reverse_edges = np.flip(np.array(edges),1)
    edges = np.concatenate([edges,reverse_edges])
    edges = np.unique(edges,axis=1) # Remove repeats
    #edges = torch.tensor(edges, dtype=torch.long)
                    
    return edges

In [51]:
y = load_pyg(x[0], x[1], x[2], folds=5, test_size=0.1)

In [52]:
y

array([[  465, 10003],
       [  465,   466],
       [  465,  2024],
       ...,
       [19156, 19676],
       [13731, 13730],
       [13743, 13742]])

In [53]:
len(y)

389266

In [54]:
torch.sparse.FloatTensor(2, 3)

tensor(indices=tensor([], size=(2, 0)),
       values=tensor([], size=(0,)),
       size=(2, 3), nnz=0, layout=torch.sparse_coo)